In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_custo_chamado"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

### DDL - Tabela de Custos
Cria tabela Silver de custos associados a cada chamado:
- **cd_custo** (PK): Identificador do registro de custo
- **cd_chamado**: FK para tb_chamado (relacionamento 1:1)
- **vl_custo**: Valor monetario com alta precisao (DECIMAL 12,10)

**Importante**: DECIMAL(12,10) garante precisao em calculos financeiros (ate 10 casas decimais).

### Configuracao
Define catalogo, schema e nome da tabela Silver de custos de chamados.

In [0]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    cd_custo BIGINT COMMENT 'Identificador do registro de custo.',
    cd_chamado BIGINT COMMENT 'Vínculo com o chamado.',
    vl_custo DECIMAL(12, 10) COMMENT 'Valor monetário do custo.',
    dt_ingestion TIMESTAMP COMMENT 'Data e hora da ingestão do registro na tabela.',
    dc_origem STRING COMMENT 'Identificação do sistema ou arquivo fonte dos dados.',
    CONSTRAINT pk_custo PRIMARY KEY (cd_custo)
)
COMMENT 'Tabela silver de custos associados a cada chamado';
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")